In [ ]:
import pandas as pd
import numpy as np
import joblib
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

#IUM - Raport końcowy

*Jakub Kordel (300233), Łukasz Reszka (300257)*

### Proces budowy modeli

W ramach zadania przygotowaliśmy 4 modele: model losowy, podstawowy model regresji liniowej, model regresji liniowej *Elastic Net* oraz model nieliniowy *SVR*. 

Budowa modeli odbyła się w następujących krokach:


*  przygotowanie danych
*  wybór modeli 
*  zaimportowanie wybranych modeli z biblioteki sklearn
*  wytrenowanie modeli na przygotowanych danych.

Przygotowane dane podzieliliśmy na dwa zbiory: trenujący i uczący. Po wytrenowaniu modeli na zbiorze trenującym oceniliśmy modele poprzez wyliczenie błędu średniokwadratowego pomiędzy przewidywanymi wartościami przez model a rzeczywistymi wartościami danej wyjściowej. 

###Przygotowanie danych

Przed przystąpieniem do uczenia modeli, otrzymane dane poddano następującemu procesowi obróbki:

1.   konwersja formatu pliku z *.jsonl* na *.csv* przy użyciu zewnętrznych narzędzi
2.   łączenie danych z plików *users.csv*, *sessions.csv* i *products.csv* w jedną tabelę zapisaną do pliku *merged.csv*. Łączenie danych realizuje specjalny program, napisany przez nas w języku *C++* (kod dołączono w *.zip*). Przy łączeniu pomijane są rekordy dotyczące sesji użytkowników, które mają braki wartości w poszczególnych atrybutach. Efekt łączenia:



In [ ]:
merged_df = pd.read_csv("merged.csv") # "merged.csv" was previously uploaded into 'content' folder on Google Colab  
merged_df.head()

,user_id,city,spent_money,year,month,bought_products,viewed_products,spent_money_next_month
0,233,Radom,0.000,2020,10,0,0,2340.090
1,235,Wrocław,346.800,2020,10,2,5,700.332
2,246,Radom,71.910,2020,10,1,8,208.250
3,247,Poznań,0.000,2020,10,0,2,1024.250
4,248,Kraków,196.964,2020,10,1,1,5338.360


3.   wczytanie danych do klasy *DataProcessor* napisanej w języku *Python*  i dalsza obróbka:


*   usunięcie zbędnych kolumn *year* oraz *user_id*
*   kodowanie wskaźnikowe atrybutów dyskretnych *month* oraz *city*
*   przeskalowanie wartości atrybutów *spent_money* i *spent_money_next_month*  z zastosowaniem logarytmu o podstawie 10
*   zaokrąglanie większych wartości atrybutów *viewed_products* i *bought_products*
*   podział danych na zbiór testowy i trenujący.

In [ ]:
class DataProcessor:
    def __init__(self, merged_framedata):
        self.merged_df = merged_framedata
        self._set_training_testing_sets()

    def get_training_testing_sets(self):
        return self.training_set, self.testing_set

    def _set_training_testing_sets(self):
        self._process_merged_data()
        self.testing_set = self.merged_df.loc[::3, :]  # every third row goes into testing set
        self.training_set = self.merged_df  # rest of data goes into training set
        self.training_set = pd.concat([self.training_set, self.testing_set, self.testing_set]).drop_duplicates(
            keep=False)

    def _process_merged_data(self):
        self.merged_df.pop('year')  # remove column 'year'
        self.merged_df.pop('user_id')  # remove column 'user_id'
        self._one_hot_encode_attribute('month')
        self._one_hot_encode_attribute('city')
        self._log10_attribute('spent_money')
        self._log10_attribute('spent_money_next_month')
        self._round_attribute('bought_products', 10)
        self._round_attribute('viewed_products', 50)

    def _one_hot_encode_attribute(self, attribute_name):
        encoded_attr_numb = len(self.merged_df[attribute_name].unique())
        one_hot_encoded_attr = pd.get_dummies(self.merged_df[attribute_name], prefix=attribute_name)
        self.merged_df = pd.concat([self.merged_df, one_hot_encoded_attr], axis=1).drop(attribute_name, axis=1)
        cols = self.merged_df.columns.tolist()
        cols = cols[-encoded_attr_numb:] + cols[:-encoded_attr_numb]  # move columns to the beginning
        self.merged_df = self.merged_df[cols]

    def _log10_attribute(self, attribute_name):
        for i, val in enumerate(self.merged_df[attribute_name]):
            if val != 0:
                self.merged_df.at[i, attribute_name] = np.log10(self.merged_df.at[i, attribute_name])

    def _round_attribute(self, attribute_name, numb_since_round):
        for i, val in enumerate(self.merged_df[attribute_name]):
            if val > numb_since_round:
                if val <= 100:
                    self.merged_df.at[i, attribute_name] = int(round(self.merged_df.at[i, attribute_name], -1))
                else:
                    self.merged_df.at[i, attribute_name] = 50 * int(round(self.merged_df.at[i, attribute_name] / 50))

W etapie 3. obróbki korzystano z biblioteki *Pandas* oraz *Numpy*.

Poniżej efekt obórki - zbiór trenujący i testowy:


In [ ]:
data_proc = DataProcessor(merged_df)
training_set, testing_set = data_proc.get_training_testing_sets()
training_set.head()

,city_Gdynia,city_Kraków,city_Poznań,city_Radom,city_Szczecin,city_Warszawa,city_Wrocław,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,spent_money,bought_products,viewed_products,spent_money_next_month
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2.540079,2,5,2.845304
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.856789,1,8,2.318585
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2.294387,1,1,3.727408
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.000000,0,0,2.015150
7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.000000,0,1,3.308980


In [ ]:
testing_set.head()

,city_Gdynia,city_Kraków,city_Poznań,city_Radom,city_Szczecin,city_Warszawa,city_Wrocław,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,spent_money,bought_products,viewed_products,spent_money_next_month
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0,0,3.369233
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0,2,3.010406
6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0,0,3.309449
9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0,0,3.794153
12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0,0,2.924079


### Modele

#### Model losowy

Zaimplementowany przez nas model losowy ma możliwość losowania wartości wyjściowych z dwóch rozkładów jednostajnego oraz normalnego.
Dla ustawienia *type = 0* używany jest rozkład normalny a dla ustawienia *type = 1* rozkład jednostajny. Po dokładnym przeanalizowaniu rozkładu wartości wydanych pieniędzy w następnym miesiącu przez użytkowników sklepu zdecydowaliśmy, że będziemy korzystać z rozkładu jednostajnego.

Parametr *zeroProb* oznacza prawdopodobieństwo z jakim wartością wylosowaną będzie 0. Takie rozwiązanie jest uzasadnione tym, że często klienci sklepu nie wydawali nic w danym miesiącu, a jeżeli już wydawali to powyżej wartości *min_val*.

Model losowy został stworzony jedynie dla celów porównawczych.

In [ ]:
class RandomModel:
    def __init__(self, mean, standard_deviation, zeroProb, min_val, max_val, type):
        self.mean = mean
        self.standard_deviation = standard_deviation
        self.zeroProb = zeroProb
        self.min_val = min_val
        self.max_val = max_val
        self.type = type

    def predict(self, x):
        predictions = []
        for i in range(0,len(x)):
            if np.random.uniform(0, 1, 1) < self.zeroProb:
                predictions.append(0.0)
            else:
                if self.type == 0:
                    predictions.append((np.random.normal(self.mean, self.standard_deviation, 1))[0])
                elif self.type == 1:
                    predictions.append(np.random.uniform(self.min_val, self.max_val, 1)[0])
        return predictions

    def evaluate(self, X, y):
        return mean_squared_error(y, self.predict(X))

#### Podstawowy model regresji liniowej

In [ ]:
from sklearn import linear_model

class LinearRegressionModel:
    def __init__(self):
        self.model = linear_model.LinearRegression()

    def train(self, X, y):
        self.model.fit(X, y)

    def save(self, filename):
        joblib.dump(self.model, filename)

    def load(self, filename):
        self.model = joblib.load(filename)

    def predict(self, x):
        return self.model.predict(x)

    def evaluate(self, X, y):
        return mean_squared_error(y, self.predict(X))

#### Model regresji liniowej *Elastic Net*

Zregularyzowany model regresji liniowej, który łączy parametry kary *L1* i *L2* z modeli *Lasso* i *Ridge*. 

In [ ]:
from sklearn.linear_model import ElasticNet

class ElasticNetModel:
    def __init__(self, a, l1_r, sel):
        self.model = ElasticNet(alpha = a, l1_ratio = l1_r, selection = sel)

    def train(self, X, y):
        self.model.fit(X, y)

    def save(self, filename):
        joblib.dump(self.model, filename)

    def load(self, filename):
        self.model = joblib.load(filename)

    def predict(self, x):
        return self.model.predict(x)

    def evaluate(self, X, y):
        return mean_squared_error(y, self.predict(X))

#### Model nieliniowy *SVR*

In [ ]:
from sklearn.svm import SVR

class SvrModel:
    def __init__(self, Cparam, epsilonParam, kerParam, gammaParam, coefParam):
        self.model = make_pipeline(StandardScaler(), SVR(C=Cparam, epsilon=epsilonParam, kernel=kerParam, gamma=gammaParam, coef0=coefParam))

    def train(self, X, y):
        self.model.fit(X, y)

    def save(self, filename):
        joblib.dump(self.model, filename)

    def load(self, filename):
        self.model = joblib.load(filename)

    def predict(self, x):
        return self.model.predict(x)

    def evaluate(self, X, y):
        return mean_squared_error(y, self.predict(X))

### Ocena modeli

Wykorzystując zbiór testowy oceniliśmy modele poprzez wyliczenie błędu średniokwadratowego ***MSE*** pomiędzy przewidywanymi przez model wartościami wydanych pieniędzy w następnym miesiącu a prawdziwą wartością wydaną przez klienta w następnym miesiącu. Poniższy skrypt wypisuje błędy średniokwadratowe dla każdego z modeli. Należy pamiętać, że wartości były przeskalowane logarytmicznie, zatem błąd też jest odpowiednio mniejszy. 


In [ ]:
x_train = training_set.loc[:, :"viewed_products"]
y_train = training_set.loc[:, "spent_money_next_month"]
x_test = testing_set.loc[:, :"viewed_products"]
y_test = testing_set.loc[:, "spent_money_next_month"]

modelSVR = SvrModel(Cparam = 1, epsilonParam = 0.5, kerParam = "rbf", gammaParam = 'scale', coefParam = 0.0)
modelEnm = ElasticNetModel(a = 0.001, l1_r = 0.8, sel = 'cyclic')
modelRandom = RandomModel(2.89358, 0.70307, float(276.0/1706.0), 1.7, 3.9, 1)
modelLrm = LinearRegressionModel()


modelSVR.train(x_train, y_train)
modelEnm.train(x_train, y_train)
modelLrm.train(x_train, y_train)

print("Błędy średniokwadratowe:")
print("Model losowy: " + str(modelRandom.evaluate(x_test, y_test)))
print("Podstawowy model liniowy: " + str(modelLrm.evaluate(x_test, y_test)))
print("Elastic Net: " + str(modelEnm.evaluate(x_test, y_test)))
print("SVR model: " + str(modelSVR.evaluate(x_test, y_test)))



Błędy średniokwadratowe:
Model losowy: 2.771819292239044
Podstawowy model liniowy: 1.3508289674672567
Elastic Net: 1.347662844167026
SVR model: 1.3253787868554674


Najgorszy okazał się model losowy co świadczy o tym, że zbudowane modele nauczyły się pewnych zależności pomiędzy danymi wejściowymi a wyjściowymi. Model *Elastic Net* okazał się niewiele lepszy od zwykłego modelu regresji liniowej ale to tylko dla bardzo małego hiperparametru *alpha*. Zwiększanie hiperparametru *alpha* tylko zwiększa błąd. Natomiast hiperparametr *l1 ratio* nie zmienia za dużo. Najlepszy wynik osiągał dla *alpha* równego 0 co sprowadzało go do zwykłego modelu liniowego. Model nieliniowy *SVR* okazał się nieznacznie lepszy od modeli liniowych.

Poniżej znajduje się skrypt wyświetlający wartości wydanych pieniedzy w następnym miesiącu przewidywane przez najlepszy model oraz rzeczywiste wartości. Z poniższego porównania widać, że przewidywania mocno odbiegają od wartości rzeczywistych. Podejrzewamy, że z otrzymanych danych trudno jest otrzymać lepsze rezultaty. 

In [ ]:
def scaleBack(tab):
  scaledValues = []
  for i in tab:
    scaledValues.append(10**i)
  return scaledValues

print("Przewidywania SVR:")
print(pd.Series(scaleBack(modelSVR.predict(x_test))).head())
print("\n")
print("Rzeczywiste wartości:")
print(pd.Series(scaleBack(y_test)).head())


Przewidywania SVR:
0    422.062996
1    375.851205
2     85.188059
3    422.062996
4    368.277812
dtype: float64


Rzeczywiste wartości:
0    2340.090
1    1024.250
2    2039.150
3    6225.200
4     839.612
dtype: float64


###Wnioski

Wnioskujemy, że dobór bardziej skomplikowanych modeli nie poprawiłby jakości przewidywań. Wynikałoby to prawdopodobnie z tego, że informacja wzajemna pomiędzy danymi wejściowymi a wyjściowymi jest zbyt mała by osiągnąć lepsze rezultaty. Dene wyznaczają granice dokładności dla której błąd średniokwadratowy wynosi 1.3. 

## Aplikacja w formie mikroserwisu

Planowana była realizacja serwisu przy pomocy frameworka *Flask* w języku Python. Zapytania do serwera przybierałyby następującą postać: http://localhost:5000/user_id/201/month/3/year/2022.

Do wykonania przewidywania potrzebna jest dana wejściowa oraz wytrenowany model. Po zapytaniu serwer ładowałby odpowiedni model z pliku i na podstawie id użytkownika i miesiąca przygotowywałby daną w wymaganej formie. 

Odpowiedzi serwer zwracałby w następującej formie:

    {

    "id_użytkownika": 201,

    "month": 3,

    "year": 2022,

    "spent_money_next_month": 979.99

    "model_type": "Elastic_net"

    }

#### Eksperyment A/B

W celu umożliwienia przeprowadzenia eksperymentu A/B, podzielilibyśmy użytkowników na trzy grupy pod względem ich id. Dla każdej z grup generowalibyśmy predykcje z innego modelu (zwykły model liniowy, *Elastic net* albo *SVR*). Przy okazji odpowiedzi serwera, na serwerze zapisywane byłyby logi z zapytania, które umożliwiłyby przeprowadzenie eksperymentu A/B.

### Czego się nauczyliśmy

- analizy danych pod kątem trenowania na nich modeli
- procesu formułowania problemu zadania modelowania 
- podstaw języka python
- zastosowania biblioteki *Pandas*, *sklearn*, *numpy*
- pracy w *Google Colab*, w którym powstał ten dokument
- obróbki danych w excelu
- dane stanowią granicę tego jaką skuteczność może osiągnąć model